In [171]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import glob
import keras.backend as K

K.clear_session() # 새로운 세션으로 시작

In [172]:
height = 128
width = 128

In [173]:
def img_read_resize(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (height, width))
    return img


In [174]:
def img_folder_read(root_dir):
    img_files = []
    nfiles = glob.glob(f"{root_dir}/*")
    nfiles = sorted(nfiles)
    for n_img in nfiles:
        try:
            img_files.append(img_read_resize(n_img))
        except:
            continue
    return img_files


In [175]:
test_root_dir = "archive/test"
test_root_dir

'archive/test'

In [176]:
data_generator = ImageDataGenerator(rescale = 1./255, validation_split=0.2)


In [177]:
train_generator = data_generator.flow_from_directory(
    directory = "archive/train",
    color_mode = 'grayscale',
    target_size = (height,width),
    batch_size = 64,
    class_mode = 'categorical',
    subset = 'training',
    shuffle = False)

Found 22968 images belonging to 7 classes.


In [178]:
train_generator.class_indices


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [179]:
val_generator = data_generator.flow_from_directory(
    directory = "archive/train",
    color_mode = 'grayscale',
    target_size =(height, width),
    class_mode = 'categorical',
    batch_size = 64,
    subset='validation',
    shuffle = False)

Found 5741 images belonging to 7 classes.


In [180]:
model = Sequential()
# 입력층
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(height, width, 1)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.8))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
# Fully-connected layer

# 출력층
model.add(Dense(7, activation='softmax'))

In [181]:
'''
# vgg 사용
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(weights = "imagenet", include_top=False, input_shape =(height, width, 3))

model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(7,activation = "softmax"))
'''

'\n# vgg 사용\nfrom tensorflow.keras.applications.vgg16 import VGG16\nvgg = VGG16(weights = "imagenet", include_top=False, input_shape =(height, width, 3))\n\nmodel = Sequential()\nmodel.add(vgg)\nmodel.add(Flatten())\nmodel.add(Dense(1024, activation = \'relu\'))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(7,activation = "softmax"))\n'

In [182]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 128)     73856     
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 128)    512       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        7

In [183]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics="accuracy")


In [184]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)


In [185]:
history = model.fit(train_generator, epochs = 2, callbacks = early_stop, validation_data = val_generator)


Epoch 1/2
359/359 [==============================] - 1507s 4s/step - loss: 12.5438 - accuracy: 0.2137 - val_loss: 8.3573 - val_accuracy: 0.2376
Epoch 2/2
359/359 [==============================] - 1530s 4s/step - loss: 1.9229 - accuracy: 0.2655 - val_loss: 2.3908 - val_accuracy: 0.2662
